In [2]:
!pip install boto3 pdfplumber

In [3]:
import os
import boto3
import json
import pdfplumber
import logging
import tempfile
from botocore.exceptions import ClientError

In [4]:
# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
# Simplificado - vai usar as credenciais da AWS CLI automaticamente
s3_client = boto3.client('s3')

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [11]:
#tem como objetivo extrair o texto de um arquivo PDF.
def extract_text_from_pdf(file_path):
    """Extrai texto de um arquivo PDF"""
    try:
        with pdfplumber.open(file_path) as pdf:
            return ''.join(page.extract_text() for page in pdf.pages if page.extract_text())
    except Exception as e:
        logger.error(f"Erro ao extrair texto do PDF: {str(e)}")
        return ""

In [7]:
def extract_sections(text_content):
    """Extrai e estrutura seções do texto"""
    sections = {}
    current_section = ""
    current_content = []

    for line in text_content.split('\n'):
        clean_line = line.strip()
        if clean_line and not clean_line.isdigit() and (clean_line.isupper() or clean_line.endswith(':')):
            if current_section:
                sections[current_section] = '\n'.join(current_content).strip()
            current_section = clean_line
            current_content = []
        elif clean_line:
            current_content.append(clean_line)

    if current_section:
        sections[current_section] = '\n'.join(current_content).strip()

    return sections

In [8]:
def process_sections(sections):
    """Processa o conteúdo das seções"""
    processed_sections = {}
    for title, content in sections.items():
        if '•' in content:
            items = [item.strip() for item in content.split('•') if item.strip()]
            processed_sections[title] = items
        else:
            processed_sections[title] = content
    return processed_sections

In [10]:
def main():
    try:
        # Configuração do bucket e arquivo
        bucket_name = 'datalake-l5'
        source_key = 'bronze/Docs-Integrações/Modelo - Proposta Tecnica - Integração telefonia MS Teams (L5 Networks).pdf'

        # Baixar o documento do S3
        logger.info(f"Lendo documento de s3://{bucket_name}/{source_key}")
        response = s3_client.get_object(Bucket=bucket_name, Key=source_key)

        # Criar arquivo temporário usando tempfile
        with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as temp_file:
            temp_pdf_path = temp_file.name
            temp_file.write(response['Body'].read())

        try:
            # Extrair texto do PDF
            text_content = extract_text_from_pdf(temp_pdf_path)
            if not text_content:
                logger.error("Nenhum texto foi extraído do PDF.")
                return

            # Extrair e processar seções
            sections = extract_sections(text_content)
            processed_content = process_sections(sections)

            # Criar estrutura JSON final
            json_data = {
                "titulo": "Proposta Técnica - Integração telefonia MS Teams",
                "empresa": "L5 Networks",
                "secoes": processed_content
            }

            # Novo caminho de salvamento no S3
            output_key = 'bronze/Docs-Integrações/Revisar Antes da Silver/' + source_key.rsplit('/', 1)[-1].rsplit('.', 1)[0] + '.json'

            # Salvar JSON no S3
            logger.info(f"Salvando JSON em s3://{bucket_name}/{output_key}")
            s3_client.put_object(
                Bucket=bucket_name,
                Key=output_key,
                Body=json.dumps(json_data, ensure_ascii=False, indent=4).encode('utf-8'),
                ContentType='application/json'
            )

            logger.info("Processamento concluído com sucesso!")

        finally:
            # Limpar arquivo temporário
            try:
                os.unlink(temp_pdf_path)
            except Exception as e:
                logger.warning(f"Erro ao remover arquivo temporário: {str(e)}")

    except ClientError as e:
        logger.error(f"Erro AWS: {str(e)}")
    except Exception as e:
        logger.error(f"Erro inesperado: {str(e)}")

if __name__ == "__main__":
    main()

INFO:__main__:Lendo documento de s3://datalake-l5/bronze/Docs-Integrações/Modelo - Proposta Tecnica - Integração telefonia MS Teams (L5 Networks).pdf
INFO:__main__:Salvando JSON em s3://datalake-l5/bronze/Docs-Integrações/Revisar Antes da Silver/Modelo - Proposta Tecnica - Integração telefonia MS Teams (L5 Networks).json
INFO:__main__:Processamento concluído com sucesso!
